In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [4]:
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

0it [00:00, ?it/s]

9920512it [00:01, 8679885.37it/s]                            


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 130966.46it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw


1654784it [00:00, 2393108.22it/s]                           
0it [00:00, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


8192it [00:00, 49917.54it/s]            

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


### **1. MNIST softmax**

learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [0]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

linear = torch.nn.Linear(784, 10, bias = True).to(device)

torch.nn.init.normal_(linear.weight) #normalize해서 initialize

Parameter containing:
tensor([[-1.2901,  0.3930, -0.1891,  ..., -0.1626, -0.3275,  0.5608],
        [-1.6757, -0.2454,  0.9221,  ...,  0.9559,  0.6160, -0.4226],
        [ 0.2444,  1.4810, -2.0906,  ...,  0.1206,  1.0044, -0.3737],
        ...,
        [ 0.6234,  1.8019, -2.7563,  ..., -0.5889, -0.5576,  0.7360],
        [-0.2871, -1.3313, -2.2248,  ...,  0.0309,  0.9180, -0.1482],
        [ 0.7678,  0.6624, -0.5362,  ...,  0.2338,  0.3688, -0.7182]],
       device='cuda:0', requires_grad=True)

In [0]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(linear.parameters(), lr = learning_rate)

In [0]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.view(-1, 28*28).to(device) #reshape input image
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = linear(X)
    cost = criterion(hypothesis, Y)
    
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  
  print('Epoch: ', '%04d'%(epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))
print('Learning Finished!')

Epoch:  0001 cost =  5.632692337
Epoch:  0002 cost =  1.694061399
Epoch:  0003 cost =  1.121575952
Epoch:  0004 cost =  0.883696675
Epoch:  0005 cost =  0.750581384
Epoch:  0006 cost =  0.662987709
Epoch:  0007 cost =  0.601501584
Epoch:  0008 cost =  0.554623663
Epoch:  0009 cost =  0.518634796
Epoch:  0010 cost =  0.489744395
Epoch:  0011 cost =  0.465924591
Epoch:  0012 cost =  0.446053177
Epoch:  0013 cost =  0.429522783
Epoch:  0014 cost =  0.414498448
Epoch:  0015 cost =  0.401699454
Learning Finished!


In [0]:
with torch.no_grad():
  X_test = mnist_test.test_data.view(-1, 28*28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = linear(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy: ', accuracy.item())

  r = random.randint(0, len(mnist_test) -1)
  X_single_data = mnist_test.test_data[r:r+1].view(-1, 28*28).float().to(device)
  Y_single_data = mnist_test.test_labels[r:r+1].to(device)

  print('Label: ',Y_single_data.item())
  single_prediction = linear(X_single_data)
  print('Prediction: ',torch.argmax(single_prediction, 1).item())

Accuracy:  0.8916999697685242
Label:  6
Prediction:  6


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


### **2. newral network**

In [0]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [0]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [0]:
#MNIST DATASET
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [0]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

##nn layers
linear1 = torch.nn.Linear(784, 256, bias = True)
linear2 = torch.nn.Linear(256, 256, bias = True)
linear3 = torch.nn.Linear(256, 10, bias = True)
relu = torch.nn.ReLU()

In [0]:
torch.nn.init.normal_(linear1.weight) #normal distribution으로 초기화
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[-1.0949, -0.9243,  0.2120,  ..., -0.2976, -1.6211, -0.6768],
        [-0.2917,  1.0723,  1.0691,  ..., -0.1451,  1.2222,  1.9022],
        [-1.0643,  0.1014,  1.4630,  ..., -0.4189,  1.4444, -1.3575],
        ...,
        [-0.6157,  0.2488,  0.3712,  ...,  0.3764, -0.6764,  1.4823],
        [ 0.4046,  0.1275, -0.4638,  ...,  1.5009, -0.9221,  0.6771],
        [ 0.1194, -0.1276, -0.4203,  ...,  1.4658, -0.3762,  1.0208]],
       requires_grad=True)

In [0]:
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)
#linear3 다음에 relu를 붙이지 않는 이유: 마지막은 softmax로 해야 하므로

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [0]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
  avg_cost = 0

  for X,Y in data_loader:
    X = X.view(-1, 28*28).to(device) #reshape
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('Epoch: ', '%04d'%(epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))  

print('Learning Finished')

Epoch:  0001 cost =  161.096359253
Epoch:  0002 cost =  39.572200775
Epoch:  0003 cost =  24.521776199
Epoch:  0004 cost =  16.799783707
Epoch:  0005 cost =  12.220030785
Epoch:  0006 cost =  9.074344635
Epoch:  0007 cost =  6.734325409
Epoch:  0008 cost =  4.987396240
Epoch:  0009 cost =  3.730864286
Epoch:  0010 cost =  2.828248262
Epoch:  0011 cost =  2.148683786
Epoch:  0012 cost =  1.609078884
Epoch:  0013 cost =  1.273704052
Epoch:  0014 cost =  0.948205769
Epoch:  0015 cost =  0.805002391
Learning Finished


In [0]:
with torch.no_grad():
  X_test = mnist_test.test_data.view(-1, 28*28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()

  print('Accuracy: ', accuracy.item())

  r = random.randint(0, len(mnist_test) - 1)
  X_single_data = mnist_test.test_data[r:r+1].view(-1, 28*28).float().to(device)
  Y_single_data = mnist_test.test_labels[r:r+1].to(device)

  print('Label: ', Y_single_data.item())
  single_prediction = model(X_single_data)
  print('Prediction: ',torch.argmax(single_prediction, 1).item())

Accuracy:  0.9490000009536743
Label:  8
Prediction:  8


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


### 3. **MNIST NN XAVIER**

In [0]:
'''기존에는 normal distribution으로 초기화했다면
이제는 xavier로 초기화, 값이 어떻게 달라지는지 보자'''

'기존에는 normal distribution으로 초기화했다면\n이제는 xavier로 초기화, 값이 어떻게 달라지는지 보자'

In [0]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [0]:
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [0]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [0]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                    batch_size = batch_size,
                                    shuffle = True,
                                    drop_last = True)

linear1 = torch.nn.Linear(784, 256, bias = True)
linear2 = torch.nn.Linear(256, 256, bias = True)
linear3 = torch.nn.Linear(256, 10, bias = True)
relu = torch.nn.ReLU()

In [0]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[ 0.0262,  0.0724, -0.1273,  ..., -0.1321, -0.0478,  0.1008],
        [-0.0840,  0.0392,  0.0247,  ...,  0.0182,  0.0752,  0.1299],
        [ 0.0158, -0.0652, -0.1156,  ...,  0.1381,  0.1440,  0.0467],
        ...,
        [-0.0754, -0.0568,  0.0534,  ...,  0.0759, -0.1003, -0.0033],
        [ 0.1499,  0.0335, -0.0746,  ..., -0.0258, -0.1090,  0.1143],
        [ 0.0889,  0.1127, -0.0531,  ..., -0.0770,  0.1412, -0.0563]],
       requires_grad=True)

In [0]:
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [0]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
  avg_cost = 0

  for X,Y in data_loader:
    
    X = X.view(-1, 28*28).to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('Epoch: ', '%04d'%(epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))
print('Learning Finished!')

Epoch:  0001 cost =  0.242789909
Epoch:  0002 cost =  0.092116676
Epoch:  0003 cost =  0.060106702
Epoch:  0004 cost =  0.042759616
Epoch:  0005 cost =  0.031430207
Epoch:  0006 cost =  0.024941990
Epoch:  0007 cost =  0.022728918
Epoch:  0008 cost =  0.015780475
Epoch:  0009 cost =  0.015942764
Epoch:  0010 cost =  0.012726117
Epoch:  0011 cost =  0.013593032
Epoch:  0012 cost =  0.011981717
Epoch:  0013 cost =  0.009693732
Epoch:  0014 cost =  0.009726852
Epoch:  0015 cost =  0.009723127
Learning Finished!


In [0]:
with torch.no_grad():
  X_test = mnist_test.test_data.view(-1, 28*28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy: ', accuracy.item())

  r = random.randint(0, len(mnist_test) - 1)
  X_single_data = mnist_test.test_data[r:r+1].view(-1, 28*28).float().to(device)
  Y_single_data = mnist_test.test_labels[r:r+1].to(device)

  print('Label: ', Y_single_data.item())
  single_prediction = model(X_single_data)
  print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy:  0.9805999994277954
Label:  7
Prediction:  7


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


### **4. MNIST NN DEEP**

In [0]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [0]:
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [0]:
learning_rate = 0.001
batch_size = 100
training_epochs = 15

In [0]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

linear1 = torch.nn.Linear(784, 512, bias = True)
linear2 = torch.nn.Linear(512, 512, bias = True)
linear3 = torch.nn.Linear(512, 512, bias = True)
linear4 = torch.nn.Linear(512, 512, bias = True)
linear5 = torch.nn.Linear(512, 10, bias = True)
relu = torch.nn.ReLU()

In [0]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0809,  0.0258, -0.0155,  ...,  0.0439,  0.0523, -0.0943],
        [-0.0008, -0.1026,  0.0759,  ...,  0.0756, -0.0981,  0.0243],
        [-0.0381,  0.0429,  0.0965,  ...,  0.0572,  0.1036,  0.0701],
        ...,
        [ 0.0221,  0.1011, -0.0943,  ...,  0.0553,  0.0010,  0.0234],
        [-0.1017,  0.0863, -0.0707,  ..., -0.0827,  0.0473,  0.0227],
        [ 0.0076,  0.0978, -0.0428,  ..., -0.1007, -0.0819,  0.0612]],
       requires_grad=True)

In [0]:
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3, relu, linear4, relu, linear5).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [0]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
  avg_cost = 0
  for X, Y in data_loader:
    X = X.view(-1, 28*28).to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('Epoch: ', '%04d'%(epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))
print('Learning Finished!')

Epoch:  0001 cost =  0.214610547
Epoch:  0002 cost =  0.090756819
Epoch:  0003 cost =  0.062855370
Epoch:  0004 cost =  0.046979416
Epoch:  0005 cost =  0.038924389
Epoch:  0006 cost =  0.035212692
Epoch:  0007 cost =  0.026331309
Epoch:  0008 cost =  0.029064538
Epoch:  0009 cost =  0.022288905
Epoch:  0010 cost =  0.021659730
Epoch:  0011 cost =  0.019422211
Epoch:  0012 cost =  0.017714908
Epoch:  0013 cost =  0.015219455
Epoch:  0014 cost =  0.018809350
Epoch:  0015 cost =  0.015362951
Learning Finished!


In [0]:
with torch.no_grad():
  X_test = mnist_test.test_data.view(-1, 28*28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy: ', accuracy.item())

  r = random.randint(0, len(mnist_test) - 1)
  X_single_data = mnist_test.test_data[r:r+1].view(-1, 28*28).float().to(device)
  Y_single_data = mnist_test.test_labels[r:r+1].to(device)

  print('Label: ', Y_single_data.item())
  single_prediction = model(X_single_data)
  print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy:  0.9772999882698059
Label:  1
Prediction:  1


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


### **5. DROPOUT**

In [0]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [0]:
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [0]:
learning_rate = 0.001
batch_size = 100
training_epochs = 15
dropout_prob = 0.3

In [0]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

linear1 = torch.nn.Linear(784, 512, bias = True)
linear2 = torch.nn.Linear(512, 512, bias = True)
linear3 = torch.nn.Linear(512, 512, bias = True)
linear4 = torch.nn.Linear(512, 512, bias = True)
linear5 = torch.nn.Linear(512, 10, bias = True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p = dropout_prob)

In [0]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[ 0.0459,  0.0725, -0.1022,  ..., -0.0240,  0.0157,  0.0907],
        [ 0.0318,  0.0065,  0.0496,  ...,  0.0996,  0.0587,  0.0070],
        [-0.0576, -0.0387,  0.0809,  ...,  0.0098,  0.0979,  0.0501],
        ...,
        [-0.1060,  0.0347, -0.0546,  ..., -0.0005, -0.1043, -0.0602],
        [-0.0566, -0.1046, -0.0300,  ...,  0.0836, -0.0670,  0.0964],
        [ 0.0307,  0.1024,  0.0221,  ..., -0.0897,  0.0981, -0.1060]],
       requires_grad=True)

In [0]:
model = torch.nn.Sequential(linear1, relu, dropout,
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4, relu, dropout,
                            linear5).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [0]:
total_batch = len(data_loader)
model.train()

for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.view(-1, 28*28).to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('Epoch: ', '%04d'%(epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))
print('Learning Finished!')

Epoch:  0001 cost =  0.309695631
Epoch:  0002 cost =  0.147655502
Epoch:  0003 cost =  0.115952045
Epoch:  0004 cost =  0.094429106
Epoch:  0005 cost =  0.081117123
Epoch:  0006 cost =  0.074026212
Epoch:  0007 cost =  0.068047471
Epoch:  0008 cost =  0.061008185
Epoch:  0009 cost =  0.058526617
Epoch:  0010 cost =  0.052880548
Epoch:  0011 cost =  0.053936947
Epoch:  0012 cost =  0.048355274
Epoch:  0013 cost =  0.046604648
Epoch:  0014 cost =  0.045599710
Epoch:  0015 cost =  0.044313796
Learning Finished!


In [0]:
with torch.no_grad():
  model.eval()
  
  X_test = mnist_test.test_data.view(-1, 28*28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy: ', accuracy.item())

  r = random.randint(0, len(mnist_test) - 1)
  X_single_data = mnist_test.test_data[r:r+1].view(-1, 28*28).float().to(device)
  Y_single_data = mnist_test.test_labels[r:r+1].to(device)

  print('Label: ', Y_single_data.item())
  single_prediction = model(X_single_data)
  print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy:  0.9817999601364136
Label:  2
Prediction:  2


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(111)
if device == 'cuda':
  torch.cuda.manual_seed_all(1)

In [0]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import random

In [0]:
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [0]:
training_epochs = 10
batch_size = 32
learning_rate = 0.01

In [0]:
train_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

test_loader = torch.utils.data.DataLoader(dataset = mnist_test,
                                          batch_size = batch_size,
                                          shuffle = False,
                                          drop_last = True)

In [0]:
#batch_normalize model
linear1 = torch.nn.Linear(784, 32, bias = True)
linear2 = torch.nn.Linear(32, 32, bias = True)
linear3 = torch.nn.Linear(32, 10, bias = True)
relu = torch.nn.ReLU()
bn1 = torch.nn.BatchNorm1d(32)
bn2 = torch.nn.BatchNorm1d(32)

#model
nn_linear1 = torch.nn.Linear(784, 32, bias = True)
nn_linear2 = torch.nn.Linear(32, 32, bias = True)
nn_linear3 = torch.nn.Linear(32, 10, bias = True)

In [0]:
bn_model = torch.nn.Sequential(linear1, bn1, relu,
                               linear2, bn2, relu,
                               linear3).to(device)

nn_model = torch.nn.Sequential(nn_linear1, relu,
                               nn_linear2, relu,
                               nn_linear3).to(device)

In [0]:
criterion = torch.nn.CrossEntropyLoss().to(device)
bn_optimizer = torch.optim.Adam(bn_model.parameters(), lr = learning_rate)
nn_optimizer = torch.optim.Adam(nn_model.parameters(), lr = learning_rate)

In [0]:
#train losses and train accuracies every epoch
train_losses = []
train_accs = []

valid_losses = []
valid_accs = []

train_total_batch = len(train_loader)
test_total_batch = len(test_loader)

In [0]:
for epoch in range(training_epochs):
  bn_model.train()

  for X, Y in train_loader:

    X = X.view(-1, 28*28).to(device)
    Y = Y.to(device)

    bn_optimizer.zero_grad()
    bn_prediction = bn_model(X)
    bn_loss = criterion(bn_prediction, Y)

    bn_loss.backward()
    bn_optimizer.step()

    nn_optimizer.zero_grad()
    nn_prediction = nn_model(X)
    nn_loss = criterion(nn_prediction, Y)

    nn_loss.backward()
    nn_optimizer.step()
  
  with torch.no_grad():
    bn_model.eval()

    bn_loss, nn_loss, bn_acc, nn_acc = 0, 0, 0, 0
    
    for i, (X, Y) in enumerate(train_loader):
      X = X.view(-1, 28*28).to(device)
      Y = Y.to(device)

      bn_prediction = bn_model(X)
      bn_correct_prediction = torch.argmax(bn_prediction, 1) == Y
      bn_loss += criterion(bn_prediction, Y)
      bn_acc += bn_correct_prediction.float().mean()

      nn_prediction = nn_model(X)
      nn_correct_prediction = torch.argmax(nn_prediction, 1) == Y
      nn_loss += criterion(nn_prediction, Y)
      nn_acc += nn_correct_prediction.float().mean()

    bn_loss, nn_loss, bn_acc, nn_acc = bn_loss / train_total_batch, nn_loss / train_total_batch, bn_acc / train_total_batch, nn_acc / train_total_batch

    train_losses.append([bn_loss, nn_loss])
    train_accs.append([bn_acc, nn_acc])
    print(
        '[Epoch %d-TRAIN] Batchnorm Loss(Acc): bn_loss: %.5f(bn_acc: %.2f) vs No Batchnorm Loss(Acc): nn_loss: %.5f(nn_acc:%.2f)'%(
            (epoch+1), bn_loss.item(), bn_acc.item(), nn_loss.item(), nn_acc.item()
        )
    )

    bn_loss, nn_loss, bn_acc, nn_acc = 0, 0, 0, 0
    for i, (X, Y) in enumerate(test_loader):
      X = X.view(-1, 28*28).to(device)
      Y = Y.to(device)

      bn_prediction = bn_model(X)
      bn_correct_prediction = torch.argmax(bn_prediction, 1) == Y
      bn_loss += criterion(bn_prediction, Y)
      bn_acc += bn_correct_prediction.float().mean()

      nn_prediction = nn_model(X)
      nn_correct_prediction = torch.argmax(nn_prediction, 1) == Y
      nn_loss += criterion(nn_prediction, Y)
      nn_acc += nn_correct_prediction.float().mean()

    bn_loss, nn_loss, bn_acc, nn_acc = bn_loss/test_total_batch, nn_loss/test_total_batch, bn_acc/test_total_batch, nn_acc/test_total_batch

    valid_losses.append([bn_loss, nn_loss])
    valid_accs.append([bn_acc, nn_acc])
    print(
        '[Epoch %d-VALID] Batchnorm Loss(Acc): bn_loss: %.5f(bn_acc: %.2f) vs No Batchnorm Loss(Acc): nn_loss:%.5f(nn_acc: %.2f)'%(
            (epoch+1), bn_loss.item(), bn_acc.item(), nn_loss.item(), nn_acc.item()
        )
    )
    print()
print('Learning Finished!')

[Epoch 1-TRAIN] Batchnorm Loss(Acc): bn_loss: 0.13220(bn_acc: 0.96) vs No Batchnorm Loss(Acc): nn_loss: 0.18864(nn_acc:0.94)
[Epoch 1-VALID] Batchnorm Loss(Acc): bn_loss: 0.14062(bn_acc: 0.96) vs No Batchnorm Loss(Acc): nn_loss:0.20625(nn_acc: 0.94)

[Epoch 2-TRAIN] Batchnorm Loss(Acc): bn_loss: 0.09267(bn_acc: 0.97) vs No Batchnorm Loss(Acc): nn_loss: 0.16408(nn_acc:0.95)
[Epoch 2-VALID] Batchnorm Loss(Acc): bn_loss: 0.10758(bn_acc: 0.97) vs No Batchnorm Loss(Acc): nn_loss:0.19956(nn_acc: 0.95)

[Epoch 3-TRAIN] Batchnorm Loss(Acc): bn_loss: 0.08396(bn_acc: 0.97) vs No Batchnorm Loss(Acc): nn_loss: 0.18723(nn_acc:0.94)
[Epoch 3-VALID] Batchnorm Loss(Acc): bn_loss: 0.10538(bn_acc: 0.97) vs No Batchnorm Loss(Acc): nn_loss:0.22912(nn_acc: 0.94)

[Epoch 4-TRAIN] Batchnorm Loss(Acc): bn_loss: 0.07921(bn_acc: 0.98) vs No Batchnorm Loss(Acc): nn_loss: 0.12901(nn_acc:0.96)
[Epoch 4-VALID] Batchnorm Loss(Acc): bn_loss: 0.09954(bn_acc: 0.97) vs No Batchnorm Loss(Acc): nn_loss:0.16911(nn_acc: 0.9